## Creating an NER Model for the Corpus

### Loading the docs

The following code loads the descriptions of each caste as doc entities from the pickled file created earlier. It then filters the descriptions to retain only those that are more than a sentence long. It then saves these to a dataframe with the relevant doc ID <span style='color:red'>(but the spaCy doc entities are converted to strings in the CSV file)</span>.

In [33]:
# Importing the required libraries
import pickle
import pandas as pd

# Loading the stored file of doc entities
with open("/Users/ilamanish/Documents/dh_ism/scripts/data/descriptions_docs.pkl", "rb") as descriptions_docs:
    docs = pickle.load(descriptions_docs)

# Loading the indices and corresponding doc entities for all descriptions longer than one sentence (hereafter referred to as long descriptions)
long_descriptions = []
ids = []

for doc_index, doc in enumerate(docs):
    sent_counter = 0
    for sent in doc.sents:
        sent_counter += 1
    if sent_counter > 1:
        ids.append(doc_index)
        long_descriptions.append(doc)

# Saving the list of sentences and corresponding doc indices as a csv
long_descriptions_df = pd.DataFrame({'doc_id': ids, 'Description': long_descriptions})
long_descriptions_df.to_csv('./data/long_descriptions.csv', index=False)

### Extracting Sentences

Sentences are individually extracted and saved to enable sentence-based analysis and modelling. A set of 1000 sample sentences is then extracted for manual annotation.

In [38]:
import pandas as pd
import random

sent_tokens_list = []
sent_text_list = []
sent_indice_list = []

for doc_id, doc in long_descriptions_df[['doc_id', 'Description']].iterrows():
    for sent in doc['Description'].sents:
        sent_tokens_list.append(sent)
        sent_text_list.append(sent.text)
        sent_indice_list.append(doc['doc_id'])

castes_dataframe = pd.read_csv('./data/castes_dataframe.csv')
castes = []
for i in sent_indice_list:
    castes.append(castes_dataframe.iloc[i,1])
    
column_names = ['Caste', 'Text Description', 'Tokenized Description']
sentences_df = pd.DataFrame(list(zip(castes, sent_text_list, sent_tokens_list)),
                  columns = column_names)

display(sentences_df)

# Save the DataFrame to a CSV file
sentences_df.to_csv('./data/all_sentences.csv', index=False)

# Extract 1000 random sentences for manual annotation
sample_sentences = random.sample(sentences_df['Text Description'].tolist(), k=1000)

# Save the sample sentences as a txt file
with open('./data/sample_sentences.txt', 'w') as file:
    # Write each element of the list to the file
    for value in sample_sentences:
        file.write(str(value) + '\n')

,Caste,Text Description,Tokenized Description
0,Acchu Tāli,A sub-division of Vāniyan.,"(A, sub, -, division, of, Vāniyan, .)"
1,Acchu Tāli,The name refers to the peculiar tāli (marriage...,"(The, name, refers, to, the, peculiar, tāli, (..."
2,Acchuvāru,"Recorded, in the Madras Census Report, 1901, a...","(Recorded, ,, in, the, Madras, Census, Report,..."
3,Acchuvāru,Treated as a sub-division of Gaudo.”,"(Treated, as, a, sub, -, division, of, Gaudo, ..."
4,Acchuvāru,"The Acchuvārus are not Oriya people, but are a...","(The, Acchuvārus, are, not, Oriya, people, ,, ..."
...,...,...,...
41866,Yōgi Gurukkal,"It is recorded, in the Gazetteer of Malabar, t...","(It, is, recorded, ,, in, the, Gazetteer, of, ..."
41867,Yōgi Gurukkal,"They perform sakti pūja in their own houses, t...","(They, perform, sakti, pūja, in, their, own, h..."
41868,Yōgi Gurukkal,"They are celebrated sorcerers and exorcists, a...","(They, are, celebrated, sorcerers, and, exorci..."
41869,Zonnala,"Zonnala, or the equivalent Zonnakūti, has been...","(Zonnala, ,, or, the, equivalent, Zonnakūti, ,..."


### Building the NER Model

#### Creating the training data

The following code uses manual annotations of **material entities** and **social relations** across 1000 sentences from the corpus to create the requisite training data. The annotation was done on tecoholic's NER Annotator: https://tecoholic.github.io/ner-annotator/.

In [ ]:
# Importing the requisite libraries
import json

# Loading the annotations from the exported JSON file
with open('./data/annotations.json', 'r') as file:
    TRAIN_DATA = json.load(file)

print(len(TRAIN_DATA))
print(TRAIN_DATA[45])

#### Training the NER Model
Code adapted from https://ner.pythonhumanities.com/03_02_train_spacy_ner_model.html 

The following code converts the training data into the binary format required in spaCy 3. It then splits the training data into a training set and a validation set and saves these in the right formats.

In [ ]:
# Importing the necessary libraries
import srsly
import typer
import spacy
import warnings
from pathlib import Path

from spacy.tokens import DocBin

# Converting the training dataset into spaCy 3 binary format
def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            # Skipping over data that is not in the required format
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

# Determining the proportion of data for the training set and validation set
train_proportion = 0.8  # Adjust this value based on your preference

# Calculating the index to split the data
split_index = int(len(TRAIN_DATA) * train_proportion)

# Splitting the data
train_data = TRAIN_DATA[:split_index]
valid_data = TRAIN_DATA[split_index:]

# Performing spaCy conversion
convert("en", train_data, "/Users/ilamanish/Documents/dh_ism/scripts/data/train.spacy")
convert("en", valid_data, "/Users/ilamanish/Documents/dh_ism/scripts/data/valid.spacy")

The following code creates a config.cfg file using the base_config.cfg file created using spaCy GUI: https://spacy.io/usage/training#quickstart.

In [ ]:
# Reformatting the base_config.cfg file into a properly formatted config.cfg file using spaCy

!python3 -m spacy init fill-config /Users/ilamanish/Documents/dh_ism/scripts/data/base_config.cfg data/config.cfg

The following code trains the NER model using the training and validation sets created earlier.

In [ ]:
# Training the spaCy model

!python3 -m spacy train data/config.cfg --output ./models/material_models/output